In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Load Data

In [2]:
df = pd.read_csv('housing_data_cleaned.csv')

In [3]:
df.dtypes

Unnamed: 0              int64
Zip_Code                int64
Dwelling_Type          object
Nr_Bedrooms             int64
Nr_Bathrooms          float64
Approx_SQFT             int64
Price_per_SqFt        float64
Dwelling_Styles        object
Year_Built              int64
Approx_Lot_SqFt         int64
Pool                   object
HOA_Fee               float64
Land_Lease_Fee         object
Clubhouse_Rec_Room     object
Basement               object
RV_Gate                object
List_Price              int64
Sold_Price              int64
Building_Style         object
Gated_Community        object
Workout_Facility       object
Garage_Spaces         float64
Carport_Spaces        float64
Loan_Type              object
Payment_Type           object
HOA_Missing             int64
Buyer_Concession      float64
Seller_Concession     float64
diff_List_Sold          int64
dtype: object

**Find if any column contains NaN values** -- looks like 'HOA_Fee' contains NaN values

In [4]:
df.columns[df.isnull().any()].tolist()

['HOA_Fee']

In [5]:
# replace all NaN values with 0 in HOA_Fee column
df['HOA_Fee'] = df['HOA_Fee'].fillna(0)  

In [6]:
# sanity check for Nan values. 
df.columns[df.isnull().any()].tolist()
# it looks good

[]

### 1. Turn categorical values in numerical using get_dummies

In [7]:
only_objects = df.select_dtypes('object').info()  # there are 12 columns of object type 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Dwelling_Type       3117 non-null   object
 1   Dwelling_Styles     3117 non-null   object
 2   Pool                3117 non-null   object
 3   Land_Lease_Fee      3117 non-null   object
 4   Clubhouse_Rec_Room  3117 non-null   object
 5   Basement            3117 non-null   object
 6   RV_Gate             3117 non-null   object
 7   Building_Style      3117 non-null   object
 8   Gated_Community     3117 non-null   object
 9   Workout_Facility    3117 non-null   object
 10  Loan_Type           3117 non-null   object
 11  Payment_Type        3117 non-null   object
dtypes: object(12)
memory usage: 292.3+ KB


In [8]:
df = pd.get_dummies(df,columns=['Dwelling_Type'], dtype='int', prefix='Type')
df = pd.get_dummies(df,columns=['Dwelling_Styles'], dtype='int',prefix='Style')
df= pd.get_dummies(df,columns=['Pool'],dtype='int', prefix='Pool')
df= pd.get_dummies(df,columns=['Land_Lease_Fee'],dtype='int', prefix='Land')
df= pd.get_dummies(df,columns=['Clubhouse_Rec_Room'],dtype='int', prefix='Club')
df= pd.get_dummies(df,columns=['Basement'], dtype='int',prefix='Base')
df= pd.get_dummies(df,columns=['RV_Gate'], dtype='int', prefix='RV_Gate')
df= pd.get_dummies(df,columns=['Building_Style'],dtype='int', prefix='Bldg_style')
df= pd.get_dummies(df,columns=['Gated_Community'],dtype='int', prefix='Gated')
df= pd.get_dummies(df,columns=['Workout_Facility'],dtype='int', prefix='Fittness')
df= pd.get_dummies(df,columns=['Loan_Type'],dtype='int', prefix='Loan')
df= pd.get_dummies(df,columns=['Payment_Type'],dtype='int', prefix='Payment')


In [9]:
df.shape

(3117, 67)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 67 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         3117 non-null   int64  
 1   Zip_Code                           3117 non-null   int64  
 2   Nr_Bedrooms                        3117 non-null   int64  
 3   Nr_Bathrooms                       3117 non-null   float64
 4   Approx_SQFT                        3117 non-null   int64  
 5   Price_per_SqFt                     3117 non-null   float64
 6   Year_Built                         3117 non-null   int64  
 7   Approx_Lot_SqFt                    3117 non-null   int64  
 8   HOA_Fee                            3117 non-null   float64
 9   List_Price                         3117 non-null   int64  
 10  Sold_Price                         3117 non-null   int64  
 11  Garage_Spaces                      3117 non-null   float

## 2. Train/Test Split

In [11]:
# manual checking for the partition sizes for 70/30 split
len(df) * .7, len(df) * .3

(2181.8999999999996, 935.0999999999999)

In [12]:
# partitioning the data into training and testing in 70/30 splits, with predicted value is "Sold_Price"

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='Sold_Price'), 
                                                    df.Sold_Price, test_size=0.3, 
                                                    random_state=47)

In [13]:
# partition sizes are consistent with the manual partitioning sizes calculated above
X_train.shape, X_test.shape

((2181, 66), (936, 66))

In [14]:
y_train.shape, y_test.shape

((2181,), (936,))

In [15]:
# Check the `dtypes` attribute of `X_train` to verify all features are numeric
X_train.dtypes

Unnamed: 0                 int64
Zip_Code                   int64
Nr_Bedrooms                int64
Nr_Bathrooms             float64
Approx_SQFT                int64
                          ...   
Payment_Fixed              int32
Payment_Graduated          int32
Payment_Interest Only      int32
Payment_Missing            int32
Payment_Other              int32
Length: 66, dtype: object

In [16]:
# Repeat this check for the test split in `X_test`
X_test.dtypes

Unnamed: 0                 int64
Zip_Code                   int64
Nr_Bedrooms                int64
Nr_Bathrooms             float64
Approx_SQFT                int64
                          ...   
Payment_Fixed              int32
Payment_Graduated          int32
Payment_Interest Only      int32
Payment_Missing            int32
Payment_Other              int32
Length: 66, dtype: object

### Determine how good the mean is as a predictor. Consider "Average Sold Price" is the best guess

In [17]:
#Calculate the mean of `y_train`
train_mean = y_train.mean()
train_mean

649888.3635946814

In [18]:
# Calculate same thing by fitting the dummy regressor on the training data and calling mean strategy
# Then print the object's `constant_` attribute and verify it's the same as the mean above

dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

DummyRegressor()

array([[649888.36359468]])

How good is this? How closely does this match, or explain, the actual values? Metrics will help answering these questions

### Metrics:  
### R-squared or coefficient of determination

In [19]:
# Manually Calculate the R^2 
def r_squared(y, ypred):
    """R-squared score.    
    Calculate the R-squared, or coefficient of determination, of the input.
    Arguments:
               y -- the observed values
               ypred -- the predicted values
    """
    ybar = np.sum(y) / len(y)            # I know I could have used np.mean(y)
    sum_sq_tot = np.sum((y - ybar)**2)   # total sum of squares error
    sum_sq_res = np.sum((y - ypred)**2)  # residual sum of squares error
    R2 = 1.0 - sum_sq_res / sum_sq_tot
    return R2

Make the predictions by creating an array "y_tr_pred" of length equal to the size of the training set with each array element equal to the single value of the mean.

In [20]:
y_tr_pred_ = train_mean * np.ones(len(y_train)) # np.ones(len(y_train)) creates an array of ones with the same length as y_train
y_tr_pred_[:5]

array([649888.36359468, 649888.36359468, 649888.36359468, 649888.36359468,
       649888.36359468])

I can obtain same thing using the sklearn dummy regressor baseline model: dumb_reg = DummyRegressor(strategy='mean')

In [21]:
y_tr_pred = dumb_reg.predict(X_train)
y_tr_pred[:5]

array([649888.36359468, 649888.36359468, 649888.36359468, 649888.36359468,
       649888.36359468])

The DummyRegressor produces exactly the same results and saves me from having to mess about broadcasting the mean (or whichever other statistic is called. It also gives an object with fit() and predict() methods as well, which can be used as conveniently as any other sklearn estimator.

In [22]:
r_squared(y_train, y_tr_pred)  # call r_squared function defined previosuly, using arguments y_train, y_tr_pred

0.0

As expected, if I use the average value as my prediction, I get an $R^2$ of zero _on the training set_. What if I use this "model" to predict unseen values from the test set? Of course, the "model" is trained on the training set; I still use the training set mean as my prediction.

In [23]:
y_te_pred = train_mean * np.ones(len(y_test))
r_squared(y_test, y_te_pred)  # call r_squared function defined previosuly, using arguments y_test, y_te_pred

-0.0078084421886748245

Performance on a test set is expected to be slightly worse than on the training set. As I'm getting an  𝑅2
  of zero on the training set, there's nowhere to go but negative!  -0.0078

Next: I use Metrics that summarise the difference between predicted and actual values which are mean absolute error and mean squared error.

This is very simply the average of the absolute errors:

$$MAE = \frac{1}{n}\sum_i^n|y_i - \hat{y}|$$

𝑦̂ 
  are our predicted values for the depended variable

In [24]:
#Calculate the MAE according to formula above

def mae(y, ypred):
    """Mean absolute error.    
    Calculate the mean absolute error of the arguments
    Arguments:
                  y -- the observed values
                  ypred -- the predicted values
    """
    abs_error = np.abs(y - ypred)
    mae = np.mean(abs_error)
    return mae

In [25]:
mae(y_train, y_tr_pred)

347574.7837341418

In [26]:
mae(y_test, y_te_pred)  

313711.9215035054

#### the MAE was less when used on test set compared to train set, which is against what was expected.  Test set is unseen so we generally expect Test MAE to be higher as it more difficult to perform well on unseen data.
 It might happen "by chance" that the test set is relatively easier (than the training set) for the model to score higher accuracy hence leading to lower Test MAE. Running 1000 different train/test splits or cross validations would be some exploration methods to further analyze. For now, I continue with sklearn metrics.
 


Mean absolute error is one of the most intuitive of all the metrics, this essentially says that, on average, expect to be off by around $313711 if the guessed Sold_Price was based on an average of known values.

### Next metric: Mean Squared Error calculated below as :   

$$MSE = \frac{1}{n}\sum_i^n(y_i - \hat{y})^2$$

MSE is a common metric (and an important one internally for optimizing machine learning models), this is simply the average of the square of the errors:

In [27]:
# Calculate the MSE according to formula above

def mse(y, ypred):
    """Mean square error.     
    Calculate the mean square error of the arguments
    Arguments:
            y -- the observed values
            ypred -- the predicted values
    """
    sq_error = (y - ypred)**2
    mse = np.mean(sq_error)
    return mse

In [28]:
mse(y_train, y_tr_pred)

407213445225.1448

In [29]:
mse(y_test, y_te_pred)

306122830945.3079

This result is consistent with the findings of MAE where against expectations, the MSE on test data was less than MSE on train data. Same conclusions apply here

To convert this back to the measurement space, take the square root, to form the root mean square error thus:

In [30]:
np.sqrt([mse(y_train, y_tr_pred), mse(y_test, y_te_pred)])

array([638132.78024651, 553283.68035331])

### sklearn.metrics: Calculate R-squared and Mean Absolute Error using sklearn.metrics

In [31]:
# R-squared
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.0, -0.0078084421886748245)

In [32]:
# Mean absolute error
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(347574.7837341418, 313711.9215035054)

In [33]:
# Mean Squarred error
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

(407213445225.1448, 306122830945.3079)

In [34]:
# train set - sklearn
r2_score(y_train, y_tr_pred)

0.0

In [35]:
# test set - sklearn
r2_score(y_test, y_te_pred)

-0.0078084421886748245

Conclusion:  The two error metrics MSE and MAE are employed. The MSE is typically larger than MAE, it penalizes larger errors more (because of the squaring operation). On the other hand MAE is more robust to the outliers sinmce it considers absolute differences. Results are conforming with expectations.

## Initial Models:  
### Steps: 
#### 1)   Scaling, 
#### 2)    Train the model on the train split
#### 3)    Make predictions using the model on both train and test splits
#### 4)    Assess model performance

First: Imputing missing feature (predictor) values
####  Impute missing values with median

In [36]:
X_defaults_median = X_train.median()
X_defaults_median

Unnamed: 0                1580.0
Zip_Code                 85207.0
Nr_Bedrooms                  3.0
Nr_Bathrooms                 2.0
Approx_SQFT               1790.0
                          ...   
Payment_Fixed                1.0
Payment_Graduated            0.0
Payment_Interest Only        0.0
Payment_Missing              0.0
Payment_Other                0.0
Length: 66, dtype: float64

Apply the imputation to both train and test splits

In [37]:
#Call `X_train` and `X_test`'s `fillna()` method, passing `X_defaults_median` as the values to use
#Assign the results to `X_tr` and `X_te`, respectively
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

In [38]:
#Call `X_train` and `X_test`'s `fillna()` method, passing `X_defaults_median` as the values to use
#Assign the results to `X_tr` and `X_te`, respectively
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

In [39]:
# As I have features measured in many different units, with numbers that vary by orders of magnitude, 
# I start off by scaling them to put them all on a consistent scale. The StandardScaler scales each feature to zero mean and 
# unit variance.

### 1) Scaling

In [40]:
# Call the StandardScaler`s fit method on `X_tr` to fit the scaler
# then use it's `transform()` method to apply the scaling to both the train and test split
# data (`X_tr` and `X_te`), naming the results `X_tr_scaled` and `X_te_scaled`, respectively
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

StandardScaler()

### 2) Train the model on the train split

In [41]:
lm = LinearRegression().fit(X_tr_scaled, y_train)

### 3)    Make predictions using the model on both train and test splits

In [42]:
#Call the `predict()` method of the model (`lm`) on both the (scaled) train and test data
#Assign the predictions to `y_tr_pred` and `y_te_pred`, respectively
y_tr_pred = lm.predict(X_tr_scaled)
y_te_pred = lm.predict(X_te_scaled)

### 4)    Assess model performance

In [43]:
# r^2 - train, test
median_r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
median_r2

(1.0, 1.0)

In [44]:
# calculate the mean absolute error scores using `sklearn`'s `mean_absolute_error` function
# as I did above for R^2
# MAE - train, test
median_mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
median_mae

(8.669231771903539e-10, 8.46933903825334e-10)

In [45]:
#And also I do the same using `sklearn`'s `mean_squared_error`
# MSE - train, test
median_mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
median_mse

(1.5054971750176088e-18, 1.4216289684009066e-18)

In [46]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(8.669231771903539e-10, 8.46933903825334e-10)

In [47]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

(1.5054971750176088e-18, 1.4216289684009066e-18)

### Pipelines

In [48]:
pipe = make_pipeline(
    SimpleImputer(strategy='median'), 
    StandardScaler(), 
    LinearRegression()
)

In [49]:
type(pipe)

sklearn.pipeline.Pipeline

In [50]:
hasattr(pipe, 'fit'), hasattr(pipe, 'predict')

(True, True)

### Fit the pipeline

In [51]:
#Call the pipe's `fit()` method with `X_train` and `y_train` as arguments
pipe.fit(X_train,y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

#### Make predictions on the train and test sets

In [52]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

#### Assess performance

In [53]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(1.0, 1.0)

### Assessing performance using cross-validation

In [54]:
cv_results = cross_validate(pipe, X_train, y_train, cv=5)

In [55]:
cv_scores = cv_results['test_score']
cv_scores

array([0.99921537, 0.99997671, 1.        , 0.99955058, 0.99922597])

In [56]:
np.round((np.mean(cv_scores) - 2 * np.std(cv_scores), np.mean(cv_scores) + 2 * np.std(cv_scores)), 2)

array([1., 1.])

### Hyperparameter search using GridSearchCV

In [57]:
#Call `pipe`'s `get_params()` method to get a dict of available parameters and print their names
#using dict's `keys()` method
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'simpleimputer', 'standardscaler', 'linearregression', 'simpleimputer__add_indicator', 'simpleimputer__copy', 'simpleimputer__fill_value', 'simpleimputer__keep_empty_features', 'simpleimputer__missing_values', 'simpleimputer__strategy', 'simpleimputer__verbose', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'linearregression__copy_X', 'linearregression__fit_intercept', 'linearregression__n_jobs', 'linearregression__positive'])

In [58]:
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}

In [59]:
lr_grid_cv = GridSearchCV(pipe, param_grid=grid_params, cv=5, n_jobs=-1)

In [60]:
lr_grid_cv.fit(X_train, y_train)

ValueError: Invalid parameter 'selectkbest' for estimator Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())]). Valid parameters are: ['memory', 'steps', 'verbose'].